In [1]:
# 1. Install required packages (only if running in a clean environment)
!pip install transformers datasets peft bitsandbytes accelerate -q

In [1]:
# 2. Imports
from datasets import load_dataset, Dataset
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig
import torch
import json
import os

In [2]:
# 3. Load the feedback data
with open("../src/feedback/finetune_data.jsonl", "r") as f:
    lines = [json.loads(l) for l in f if l.strip()]

dataset = Dataset.from_list(lines)
print("Loaded examples:", len(dataset))


Loaded examples: 20


In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id = "google/flan-t5-small"

# Load tokenizer and model (no quantization, no bnb)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# Set padding token (important for training)
tokenizer.pad_token = tokenizer.eos_token

In [4]:
# 5. Apply LoRA
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q", "v"],  # T5 uses different layer names
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM,
)

model = get_peft_model(model, lora_config)

bitsandbytes library load error: dlopen(/opt/anaconda3/envs/allyin/lib/python3.12/site-packages/bitsandbytes/libbitsandbytes_cpu.dylib, 0x0006): tried: '/opt/anaconda3/envs/allyin/lib/python3.12/site-packages/bitsandbytes/libbitsandbytes_cpu.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/anaconda3/envs/allyin/lib/python3.12/site-packages/bitsandbytes/libbitsandbytes_cpu.dylib' (no such file), '/opt/anaconda3/envs/allyin/lib/python3.12/site-packages/bitsandbytes/libbitsandbytes_cpu.dylib' (no such file)
Traceback (most recent call last):
  File "/opt/anaconda3/envs/allyin/lib/python3.12/site-packages/bitsandbytes/cextension.py", line 287, in <module>
    lib = get_native_library()
          ^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/allyin/lib/python3.12/site-packages/bitsandbytes/cextension.py", line 274, in get_native_library
    dll = ct.cdll.LoadLibrary(str(binary_path))
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/allyin/lib/py

In [5]:
# 6. Format dataset for SFT
def format_prompt(example):
    inputs = tokenizer(example["prompt"], truncation=True, padding="max_length", max_length=512)
    targets = tokenizer(example["completion"], truncation=True, padding="max_length", max_length=512)

    return {
        "input_ids": inputs.input_ids,
        "attention_mask": inputs.attention_mask,
        "labels": targets.input_ids,
    }

tokenized_dataset = dataset.map(format_prompt, remove_columns=dataset.column_names)

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [7]:
# 7. Define training args
training_args = TrainingArguments(
    output_dir="../models/lora_adapter",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=2,
    save_strategy="no",  # Or "epoch" if you want to save
    learning_rate=2e-4,
    fp16=False,          # Set True only if your GPU supports it
    bf16=True,           # Safer on newer hardware, esp. MPS
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

# 8. Train
trainer.train()

/var/folders/fc/t8b4fmzd1vn4x8ky425rp7l40000gn/T/ipykernel_30282/2507841326.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,3.144500


TrainOutput(global_step=10, training_loss=3.144534873962402, metrics={'train_runtime': 7.5119, 'train_samples_per_second': 5.325, 'train_steps_per_second': 1.331, 'total_flos': 7477898772480.0, 'train_loss': 3.144534873962402, 'epoch': 2.0})

In [8]:
# 9. Save adapter
model.save_pretrained("../models/lora_adapter")
tokenizer.save_pretrained("../models/lora_adapter")

print("Fine-tuning simulation complete. LoRA adapter saved.")

Fine-tuning simulation complete. LoRA adapter saved.
